## Le tour de France des musées

Tu viens d'être recruté en tant que stagiaire Data Analyst par le Ministère de la Culture. Le précédent stagiaire avait réussi à récolter le maximum d'informations sur les musées français.  
Comme première mission, ton manager te demande d'analyser avec Python le fichier csv obtenu et d'essayer de trouver des informations intéressantes dans ce fichier.  

### 1. Télécharger et ouvrir le fichier CSV

In [1]:
import pandas as pd
import numpy as np
import re
from tabulate import tabulate

museums = pd.read_csv('liste-des-musees-de-france-2021.csv', encoding='latin-1', delimiter=";")
display(museums)

,osm_id,name,number,street,postal_code,city,country,country_code,lat,lon,website,email,phone,fax,tags,description,date_added,wikidata
0,4540234591,3 bis f,109,Avenue du Petit Barthélémy,13617.0,Aix-en-Provence,France,fr,43.521207,5.439875,http://www.3bisf.com,contact@3bisf.com,3.344216e+10,NaN,osm:museum;art:art contemporain,NaN,"44116,0076388889",NaN
1,4388745689,A casa di Losari,NaN,T 30,20226.0,Belgodère,France,fr,42.638732,9.007660,NaN,NaN,NaN,NaN,osm:museum,NaN,"43338,9393981481",NaN
2,4426176194,À l'homme volant,35,Grande Rue,72300.0,Sablé-sur-Sarthe,France,fr,47.839656,-0.334043,NaN,NaN,3.324395e+10,NaN,osm:museum;type:centre d'interpretation,NaN,"42644,4636574074",NaN
3,7652598615,Abbadiale - Maison des arts,NaN,Rue du Gabizos,65400.0,Arras-en-Lavedan,France,fr,42.990826,-0.128057,https://www.abbadiale.fr/,NaN,3.363317e+10,NaN,osm:museum,NaN,"44120,0466782407",NaN
4,121580330,Abbaye,NaN,Rue de l'Abbaye,39210.0,Château-Chalon,France,fr,46.753866,5.624296,NaN,NaN,NaN,NaN,osm:museum;art:architecture,NaN,"43102,5491898148",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3779,1348524503,Vivarium du Moulin,NaN,Rue Principale,68610.0,Thann-Guebwiller,France,fr,47.942258,7.149613,NaN,NaN,NaN,NaN,osm:museum,NaN,"40747,8717476852",NaN
3780,577287841,Voyage au Temps des impressionnistes,NaN,Rue de Léry,95430.0,Auvers-sur-Oise,France,fr,49.072057,2.166167,NaN,NaN,NaN,NaN,osm:museum,NaN,"41141,7272222222",NaN
3781,145062184,Wines and Spirits Museum,NaN,Corniche Bonaparte,83150.0,Bandol,France,fr,43.128031,5.749484,NaN,NaN,NaN,NaN,osm:museum,NaN,"42367,7135069444",NaN
3782,82988725,Xanxotea,NaN,La Place,64240.0,Isturits,France,fr,43.364834,-1.203580,http://www.xanxotea.com/,NaN,NaN,NaN,osm:museum,Musée ethnographique Xanxotea pour découvrir l...,"44129,580775463",NaN


### 2. Répondre aux questions du manager

In [2]:
# Combien y-a-t-il de musées en France métropolitaine ?
total_museums = len(museums)
print(f'Il y a {total_museums} musées en France métropolitaine.')

Il y a 3784 musées en France métropolitaine.


In [3]:
# Dans quelle(s) ville(s) y-a-t-il le plus de musées ?
city_counts = museums.groupby('city').size().reset_index(name='count')
city_counts = city_counts.sort_values(by='count', ascending=False)
most_museums_city = city_counts.iloc[0]['city']
num_museums = city_counts.iloc[0]['count']
print(f"La ville avec le plus de musées est {most_museums_city}, avec {num_museums} musées.")

La ville avec le plus de musées est Paris, avec 175 musées.


In [4]:
# Quel est le nombre moyen de musées par ville ?
avg_museums_per_city = city_counts['count'].mean()
print('Nombre moyen de musées par ville:', avg_museums_per_city)

Nombre moyen de musées par ville: 1.647930283224401


In [5]:
# Quel est le nombre médian de musées par ville ?
median_museums_per_city = city_counts['count'].median()
print('Nombre médian de musées par ville:', median_museums_per_city)

Nombre médian de musées par ville: 1.0


In [6]:
# Comment sont répartis les musées par type (en pourcentage) ?
tags = museums['tags'].dropna()
tags = tags[tags != 'osm:museum']
types = tags.str.findall(r'type:(.*?)(?:;|$)').explode().str.strip().str.replace('é', 'e')
types = types.str.replace('ecomuseum', 'ecomusee')
types_count = types.value_counts(normalize=True) * 100

table = []

for index, value in types_count.items():
    table.append([index.replace('type:', ''), f"{value:.2f}%"])
print('Répartition des musées par type :')
print(tabulate(table, headers=['Type', 'Percentage']))

Répartition des musées par type :
Type                           Percentage
-----------------------------  ------------
ecomusee                       27.07%
musee technique et industriel  14.29%
musee d'histoire               10.95%
musee de beaux-arts            8.64%
musee archeologique            7.41%
musee d'art                    7.41%
maison d'artiste               4.49%
musee d'arts populaires        3.95%
musee d'histoire naturelle     3.88%
centre d'interpretation        3.13%
musee litteraire               2.24%
musee de sciences              1.56%
centre d'art                   1.36%
fondation                      1.22%
musee d'artiste                1.02%
atelier d'artiste              0.54%
bibliotheque                   0.27%
musee ethnographique           0.20%
musee d'arts populaires        0.14%
musee d'architecture           0.07%
maison litteraire              0.07%
musee d'art populaire          0.07%


In [7]:
# Combien y-a-t-il de musées dont le nom commence par "Château" ?
num_chateau_museums = museums['name'].str.startswith('Château').sum()
print(f"Il y a {num_chateau_museums} musées qui commencent par 'Château'.")

Il y a 57 musées qui commencent par 'Château'.


In [8]:
# Pour combien de musées dispose-t-on de l'adresse du site web ?
num_websites = museums['website'].count()
print(f"Il y a {num_websites} musées qui disposent d'une adresse de site web.")

Il y a 1636 musées qui disposent d'une adresse de site web.


In [9]:
# Quel département français possède le plus de musées sur son territoire ?
museums['postal_code'] = museums['postal_code'].astype(str)
museums['dept_code'] = museums['postal_code'].str[:2]
dept_counts = museums['dept_code'].value_counts()
max_dept = dept_counts.index[0]
max_dept_count = dept_counts[max_dept]
print(f"Le département avec le plus de musées est celui du {max_dept} avec {max_dept_count} musées.")

Le département avec le plus de musées est celui du 75 avec 180 musées.


In [10]:
# Quel département français possède le moins de musées sur son territoire ?
min_dept = dept_counts.index[-1]
min_dept_count = dept_counts[min_dept]
print(f"Le département avec le moins de musées est celui du {min_dept} avec {min_dept_count} musées.")

Le département avec le moins de musées est celui du 98 avec 8 musées.


In [11]:
# Combien de musées ont "Napoléon" dans leur nom ?
napoleon_count = museums['name'].str.contains('Napoléon').sum()
print(f"Il y a {napoleon_count} musées avec 'Napoléon' dans leur nom.")

Il y a 4 musées avec 'Napoléon' dans leur nom.


### 3. Axes d'analyse supplémentaires

Donner d'autres sources contenant des informations supplémentaires qui peuvent être ajoutées au fichier initial :

- La base "Joconde" qui contient le catalogue des collections de musées de France :  
https://cour-de-france.fr/actualites/liens-utiles/bases-de-donnees-publiees-sur-des-sites-tiers/article/base-joconde-catalogue-des-collections-des-musees-de-france?lang=fr  
Créée en 1975, JOCONDE donne aujourd’hui accès (fin mars 2008) à plus de 366.000 notices d’objets, dont 200.000 sont illustrées par une ou plusieurs images. Ces notices proviennent de plus de 282 musées différents (affectataires ou dépositaires) qui conservent des collections d’archéologie, de beaux-arts, d’arts décoratifs, d’ethnologie, d’histoire ou encore de sciences et techniques.

- La base Muséophile qui répertorie les musées de France :  
https://data.culture.gouv.fr/explore/dataset/musees-de-france-base-museofile/information/  
La base Muséofile répertorie les musées bénéficiant de l'appellation « musée de France » au sens du Code du Patrimoine (1220 environ). Elle est accessible sur la Plateforme Ouverte du Patrimoine. Les données de cette base sont liées à celles de la base Joconde, catalogue collectif des collections des musées de France.

- La base des fréquentations des musées de France :  
https://data.culture.gouv.fr/explore/dataset/frequentation-des-musees-de-france/information/  
Fréquentation totale, payante et gratuite dans les Musées de France de 2001 à 2020.
